In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn.functional as F
from langchain import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.schema import Document
from datasets import load_from_disk
from peft import LoraConfig, get_peft_model
from transformers import (AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          AutoTokenizer,
                          GenerationConfig,
                          pipeline)

from langchain.llms import HuggingFacePipeline


bin C:\ProgramData\Miniconda\envs\qlora\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


1. https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html#langchain.llms.huggingface_pipeline.HuggingFacePipeline
2. https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
3. https://huggingface.co/docs/transformers/main_classes/pipelines

In [2]:
model_path = "llms/vicuna-13b-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    load_in_8bit=True,
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_quant_type="nf4",
    #     bnb_4bit_compute_dtype=torch.float16
    # ),
    trust_remote_code=True
)

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=GenerationConfig(
                max_new_tokens=10,
                top_p=0.0,
                temperature=0.0
    )
    # max_new_tokens=10,
    # temperature=0.0,
    # top_p=0.0
)

hf = HuggingFacePipeline(pipeline=pipe)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
user_instruct = 'Answer the question directly based on the context below. It is important for me that you answer the question with one single word: "Yes", "No", or "Maybe". If the question cannot be answered using the information provided answer with "No".'

context = '''- Objective: To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer.
- Methods: Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of cisplatin monotherapy after cystectomy. This study had a negative result. The outcome of these 35 patients (protocol group) was compared with an age- and tumor-stage-matched cohort (matched group; n = 35) who also underwent cystectomy during the same period, but were not entered into the SAKK study, as well as the remaining 57 patients treated during the study period for the same indication (remaining group).
- Results: Median overall survival decreased from 76.3 months in the protocol group to 52.1 months in the matched group and to 20.3 months in the remaining group. The respective times of median recurrence-free survival were 67.2, 16.0, and 9.4 months. Tumor progression occurred in 46% of the protocol group compared with 69% in the matched group and 65% in the remaining group (P<.05). Cancer-related death was noted in 40% of the protocol group, 57% in the matched group, and 56% in the remaining group.'''

query = "Question: Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?"

context_template = '''- Objective: [{objective}]\n- Methods: [{methods}]\n- Results: [{results}]'''
query_template = '''Question: {query}'''

In [4]:
qa_template = """{start}{instruction_begin}{system_begin}{system_msg}{system_end}
{user}{user_instruct}\n
{query}\n
Context: \n{context}{instruction_end}{assistant}{answer}{end}"""


In [5]:
def prompt_formatter(template, context, user_instruct, query, answer="", system_msg="", model="vicuna", prompt_type="eval"):

    system_msg = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions." if system_msg =="" else system_msg
    vicuna_kwargs = {
        "start": "",
        "instruction_begin": "",
        "system_begin": "",
        "system_end": "\n",
        "user": "USER: ",
        "instruction_end": "\n",
        "assistant": "\nASSISTANT: ",
        }

    llama_kwargs = {
        "start": "<s>",
        "instruction_begin": "[INST] ",
        "system_begin": "<</SYS>>\n",
        "system_end": "\n<</SYS>>\n",
        "user": "",
        "instruction_end": " [/INST] ",
        "assistant": "",
        }

    prompt_template = PromptTemplate(
        input_variables=[
            "start", "end", "instruction_begin", "instruction_end",
            "system_begin", "system_end", "system_msg", "user",
            "user_instruct", "context", "query", "assistant", "answer", "end"
        ],
        template=template
    )

    current_kwargs = vicuna_kwargs if model == "vicuna" else llama_kwargs
    prompt = prompt_template.format(
        context=context,
        query=query,
        system_msg=system_msg,
        user_instruct=user_instruct,
        answer="" if prompt_type == "eval" else answer,
        end="" if prompt_type == "eval" else " </s>",
        **current_kwargs
    )

    return prompt

In [6]:
print(prompt_formatter(qa_template, context, user_instruct, query, model="vicuna", prompt_type="train"))


A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER: Answer the question directly based on the context below. It is important for me that you answer the question with one single word: "Yes", "No", or "Maybe". If the question cannot be answered using the information provided answer with "No".

Question: Is eligibility for a chemotherapy protocol a good prognostic factor for invasive bladder cancer after radical cystectomy?

Context: 
- Objective: To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer.
- Methods: Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of 

In [7]:
simple_answer_instruct = """Answer the question directly based on the context below. It is important for me that you answer the question with one single word: "Yes", "No", or "Maybe". If the question cannot be answered using the information provided answer with "No"."""
complex_answer_instruct = """Answer the question directly based on the context below. It is important for me that you answer the question first with "Yes", "No", or "Maybe", and then explain your answer. If the question cannot be answered using the information provided answer with "I do not know"."""
keyword_generation_instruct = """Generate keywords that are relevant to the context below."""


In [8]:
print(complex_answer_instruct)


Answer the question directly based on the context below. It is important for me that you answer the question first with "Yes", "No", or "Maybe", and then explain your answer. If the question cannot be answered using the information provided answer with "I do not know".


In [9]:
print(hf(prompt_formatter(qa_template, context, user_instruct, query, model="vicuna", prompt_type="eval")))


Yes.
